In [10]:
import os
import re
import torch
from glob import glob
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

In [11]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
model = GPT2LMHeadModel.from_pretrained("gpt2-large")

In [30]:
dataset_path = "../input/data/nomic_ai_parquet1.parquet_0.parquet"
files = None
if os.path.exists(dataset_path):
    if os.path.isdir(dataset_path):
        files = glob.glob(os.path.join(dataset_path, "_*clean.parquet"))
    else:
        files = [dataset_path]

    print(f"Reading files {files}")

dataset = load_dataset("parquet", data_files=files, split="train")

dataset = dataset.train_test_split(test_size=.05, seed=42)

train_dataset, val_dataset = dataset["train"], dataset["test"]

print(train_dataset)
print(val_dataset)

Reading files ['../input/data/nomic_ai_parquet1.parquet_0.parquet']


Using custom data configuration default-a88804c9e41564b9
Reusing dataset parquet (C:\Users\DongViet\.cache\huggingface\datasets\parquet\default-a88804c9e41564b9\0.0.0\7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)
Loading cached split indices for dataset at C:\Users\DongViet\.cache\huggingface\datasets\parquet\default-a88804c9e41564b9\0.0.0\7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8\cache-db359562c2683520.arrow and C:\Users\DongViet\.cache\huggingface\datasets\parquet\default-a88804c9e41564b9\0.0.0\7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8\cache-76022d614956306e.arrow


Dataset({
    features: ['prompt', 'response', 'source'],
    num_rows: 948
})
Dataset({
    features: ['prompt', 'response', 'source'],
    num_rows: 50
})


In [37]:
train_dts = ""
val_dts = ""

for row in train_dataset:
    prompt = f"[Q]: {row['prompt']}"
    new_line = "\n"
    response = f"[A]: {row['response']}"
    string = prompt + new_line + response
    train_dts += string
    
for row in val_dataset:
    prompt = f"[Q]: {row['prompt']}"
    new_line = "\n"
    string = prompt + new_line + response
    val_dts += string
    
train_dts

'[Q]: <p>I am trying to get some basic code running. I am using a SSIS script component (source type). I have set up the component using the code in <a href="http://technet.microsoft.com/en-us/library/ms136060(v=sql.105).aspx" rel="nofollow">http://technet.microsoft.com/en-us/library/ms136060(v=sql.105).aspx</a> (section: ADO.NET Source Example). However the following line is gnerating an error: </p>\n\n<p>connMgr = this.Connections. MyADONETConnection;</p>\n\n<p>\'Connections\' does not contain a definition for \'MyADONETConnection\' and no extension method \'MyADONETConnection\' accepting a first argument of type \'Connections\' could be found (are you missing a using directive or an assembly reference?)</p>\n\n<p>I have set up the connection and output columns as per the instructions.\nAny thoughts appreciated.\nJohnny</p>\n[A]: It seems like there may be an issue with the name of the connection manager. Double-check that the name of the connection manager in your SSIS package match

In [38]:
# Save the training and validation data as text files
with open("train.txt", "w") as f:
    f.write(train_dts)
with open("val.txt", "w") as f:
    f.write(val_dts)

# Prepare the dataset
train_dataset = TextDataset(tokenizer=tokenizer, file_path="train.txt", block_size=128)
val_dataset = TextDataset(tokenizer=tokenizer, file_path="val.txt", block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

UnicodeEncodeError: 'charmap' codec can't encode character '\ufe55' in position 14168: character maps to <undefined>

In [21]:
model_output_path = "../output"

In [22]:
training_args = TrainingArguments(
    output_dir=model_output_path,
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=100,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
)

trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

trainer.train()
trainer.save_model(model_output_path)
    
# Save the tokenizer
tokenizer.save_pretrained(model_output_path)

ValueError: num_samples should be a positive integer value, but got num_samples=0